# AML 1

## Task 1

In [ ]:
%pip install torchsummary

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
batch_size = 32

# Dataset import as in Ptytorch docs
ds_train = datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor(),)
ds_test = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor(),)

train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False)

In [ ]:
# Defining the labels from datasetg - taken from kaggle
labels = {
    0: 'T-shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot',
}

In [ ]:
# Selecting the right device to perform the computations on
device ='cuda' if torch.cuda.is_available else 'cpu'
print(f'Using {device} device')

Using cuda device


In [ ]:
# Definition of the model

class FashionMnistClassifier(nn.Module):
  def __init__(self):
    super(FashionMnistClassifier, self).__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.drop1 = nn.Dropout2d(0.25)

    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.drop2 = nn.Dropout2d(0.25)

    self.flatten = nn.Flatten()

    self.linear_relu_stack = nn.Sequential(
            nn.Linear(7*7*64, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

  def forward(self, x):
    out = self.conv_block_1(x)
    out = self.drop1(out)
    out = self.conv_block_2(out)
    out = self.drop2(out)
    out = self.linear_relu_stack(self.flatten(out))
    return out


In [ ]:
model = FashionMnistClassifier().to(device)
print(model.parameters())

<generator object Module.parameters at 0x7f2d868f0f90>


In [ ]:
# Settings
learning_rate = 1e-3
epochs = 5

loss = nn.CrossEntropyLoss()#nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [ ]:
def training_epoch(dataloader, model, loss, opt):
  model.train() # Setting model into train mode
  samples = len(dataloader)*dataloader.batch_size
  processed = 0
  batch_num = 0

  for data, label in dataloader:
    data = data.to(device)
    label = label.to(device)

    opt.zero_grad()

    predictions = model(data)
    loss_result = loss(predictions, label)
    loss_result.backward()

    opt.step()

    processed += len(data)
    if batch_num % 100 == 0:
      cur_loss = loss_result.item()
      print(f'loss:{cur_loss:>7f} [{processed:>5d}/{samples:>5d}]')

    batch_num += 1

In [ ]:
def evaluate(dataloader, model, loss):
  samples = len(dataloader.dataset)
  n_batches = len(dataloader)
  model.eval() # Disenables storing training data

  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, label in dataloader:
      data = data.to(device)
      label = label.to(device)

      predictions = model(data)

      loss_val = loss(predictions, label).item()
      correct += (predictions.argmax(1) == label).type(torch.float).sum().item()
      test_loss += loss_val
  test_loss /= n_batches
  accuracy = correct/samples

  return accuracy, test_loss, correct #, cm


#     correct = 0
#     with torch.no_grad():
#         for data, label in dataloader:
#             data, label = data.to(device), label.to(device)
#             predictions = model(data)
#             correct += (predictions.argmax(1) == label).type(torch.float).sum().item()
#     accuracy = correct / len(dataloader.dataset)



In [ ]:
def run_training(train, test, model, loss, opt, epochs):

  for e in range(epochs):
    print(f"# Epoch {e}")
    training_epoch(train, model, loss, opt)
    train_accuracy, train_loss, train_correct = evaluate(train, model, loss)
    test_accuracy, test_loss, test_correct = evaluate(test, model, loss)
    print(f"train_loss = {train_loss}, train_acc = {train_accuracy}, test_loss = {test_loss}, test_acc = {test_accuracy}")


In [ ]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
              ReLU-3           [-1, 32, 28, 28]               0
         MaxPool2d-4           [-1, 32, 14, 14]               0
         Dropout2d-5           [-1, 32, 14, 14]               0
            Conv2d-6           [-1, 64, 14, 14]          18,496
       BatchNorm2d-7           [-1, 64, 14, 14]             128
              ReLU-8           [-1, 64, 14, 14]               0
         MaxPool2d-9             [-1, 64, 7, 7]               0
        Dropout2d-10             [-1, 64, 7, 7]               0
          Flatten-11                 [-1, 3136]               0
           Linear-12                  [-1, 512]       1,606,144
             ReLU-13                  [-1, 512]               0
           Linear-14                  [

In [ ]:
run_training(train_loader, test_loader, model, loss, optimizer, 20)

# Epoch 0
loss:2.327907 [   32/60000]
loss:2.262675 [ 3232/60000]
loss:2.177407 [ 6432/60000]
loss:2.074695 [ 9632/60000]
loss:1.940964 [12832/60000]
loss:1.825862 [16032/60000]
loss:1.653764 [19232/60000]
loss:1.525925 [22432/60000]
loss:1.270920 [25632/60000]
loss:1.102172 [28832/60000]
loss:1.189249 [32032/60000]
loss:1.206663 [35232/60000]
loss:1.094579 [38432/60000]
loss:0.931631 [41632/60000]
loss:0.952184 [44832/60000]
loss:0.775432 [48032/60000]
loss:0.980141 [51232/60000]
loss:0.950272 [54432/60000]
loss:0.508294 [57632/60000]
train_loss = 0.7308023591518402, train_acc = 0.7579, test_loss = 0.7367126640801231, test_acc = 0.7553
# Epoch 1
loss:1.026207 [   32/60000]
loss:0.640689 [ 3232/60000]
loss:0.980416 [ 6432/60000]
loss:0.625771 [ 9632/60000]
loss:0.909272 [12832/60000]
loss:0.789165 [16032/60000]
loss:0.731916 [19232/60000]
loss:0.656753 [22432/60000]
loss:0.519451 [25632/60000]
loss:0.768047 [28832/60000]
loss:0.474810 [32032/60000]
loss:0.643805 [35232/60000]
loss:0.56

## Task 2

In [ ]:
class AutoEncoder(torch.nn.Module):
  def __init__(self, input_size):
    super(AutoEncoder, self).__init__()
    self.encoder = torch.nn.Sequential(
        torch.nn.Linear(input_size, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64,32),
        torch.nn.ReLU(),
        torch.nn.Linear(32,16)
    )

    self.decoder = torch.nn.Sequential(
        torch.nn.Linear(16,32),
        torch.nn.ReLU(),
        torch.nn.Linear(32,64),
        torch.nn.ReLU(),
        torch.nn.Linear(64,128),
        torch.nn.ReLU(),
        torch.nn.Linear(128,input_size),
        torch.nn.ReLU()
    )

  def forward(self, x):
    out = self.encoder(x)
    out = self.decoder(out)
    return out


In [ ]:
ds_train = datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor())
ds_test = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor())
batch_size=32
train_dataloader = DataLoader(ds_train,batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(ds_test,batch_size=batch_size, shuffle=True)

model = AutoEncoder(28*28)


optimizer = torch.optim.Adam(model.parameters(),
                             lr = 1e-1,
                             weight_decay = 1e-8)

loss_f = torch.nn.MSELoss()

# Selecting the right device to perform the computations on
device ='cuda' if torch.cuda.is_available else 'cpu'
print(f'Using {device} device')

model = model.to(device)

Using cuda device


In [ ]:
def one_epoch_train(model, optimizer, loss, train_dataloader, device):
  model.train()
  processed_images = 0
  current_batch_index = 0
  samples = len(train_dataloader.dataset)
  for image, _ in train_dataloader:
    image = image.to(device)
    image = image.reshape(-1,28*28)

    reconstructed_image = model(image)
    computed_loss = loss(reconstructed_image, image)
    computed_loss.backward()

    optimizer.step()

    processed_images += len(image)
    if current_batch_index % 100 == 0:
      current_loss_value = computed_loss.item()
      print(f'loss:{current_loss_value:>7f} [{processed_images:>5d}/{samples:>5d}]')
    current_batch_index +=1


In [ ]:
def evaluate(model, loss, dataloader, device):
  model.eval()
  samples = len(dataloader.dataset)
  n_batches = len(dataloader)
  total_loss = 0
  samples = len(dataloader.dataset)
  for image, _ in dataloader:
    image = image.to(device)
    image = image.reshape(-1, 28*28)

    reconstructed_image = model(image)
    computed_loss = loss(reconstructed_image, image)
    total_loss+=computed_loss.item()
  return total_loss/n_batches

In [ ]:
def run_training(train, test, model, loss, opt, epochs):

  for e in range(epochs):
    print(f"# Epoch {e}")
    one_epoch_train(model,opt, loss, train, device)
    train_loss = evaluate(model, loss, train,device)
    test_loss = evaluate(model, loss, test,device)
    print(f"train_loss = {train_loss}, test_loss = {test_loss}")
run_training(train_dataloader, test_dataloader, model, loss_f, optimizer, 20)

# Epoch 0
loss:0.230689 [   32/60000]
loss:0.221271 [ 3232/60000]
loss:0.209455 [ 6432/60000]
loss:0.208472 [ 9632/60000]
loss:0.204828 [12832/60000]
loss:0.232972 [16032/60000]
loss:0.181306 [19232/60000]
loss:0.238475 [22432/60000]
loss:0.192490 [25632/60000]
loss:0.245523 [28832/60000]
loss:0.242361 [32032/60000]
loss:0.213575 [35232/60000]
loss:0.191897 [38432/60000]
loss:0.214855 [41632/60000]
loss:0.205197 [44832/60000]
loss:0.222642 [48032/60000]
loss:0.158204 [51232/60000]
loss:0.176766 [54432/60000]
loss:0.211015 [57632/60000]
train_loss = 0.20644534345467885, test_loss = 0.20652934332815603
# Epoch 1
loss:0.228290 [   32/60000]
loss:0.225174 [ 3232/60000]
loss:0.199176 [ 6432/60000]
loss:0.209543 [ 9632/60000]
loss:0.244000 [12832/60000]
loss:0.212643 [16032/60000]
loss:0.217909 [19232/60000]
loss:0.178870 [22432/60000]
loss:0.179798 [25632/60000]
loss:0.197164 [28832/60000]
loss:0.220349 [32032/60000]
loss:0.237585 [35232/60000]
loss:0.243746 [38432/60000]
loss:0.230232 [416